# 0. 라이브러리 불러오기

In [1]:
import mido
from mido import MidiFile, MidiTrack
import numpy as np
import pandas as pd
import editdistance
import os

# 1. 동일한 시간에 동시에 누르는 건반에 대해서 데이터 추출하기

In [2]:
def midi_to_dataframe(file_path):
    mid = mido.MidiFile(file_path)

    # DataFrame 초기화
    columns = ['Sec', 'Ticks', 'msg_type', 'Channel', 'Notes', 'Velocities', 'Pedal', 'Tempo']
    midi_df = pd.DataFrame(columns=columns)

    # 노트와 velocity, 페달, 템포, 틱스, 시간을 저장할 리스트 초기화
    current_notes = []
    current_velocities = []
    current_pedal = None
    current_tempo = None
    current_ticks = None
    current_time = 0
    current_msg_type = []

    for msg in mid:
        if msg.type in ['note_on', 'note_off']:
            # 동일한 시간에 친 노트들을 리스트로 묶어서 저장
            if msg.time > 0:
                midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, msg.channel, current_notes, 
                                                             current_velocities, current_pedal, current_tempo]], columns=columns)], ignore_index=True)
                current_notes = []
                current_velocities = []
                current_msg_type = []

            current_notes.append(msg.note)
            current_velocities.append(msg.velocity)
            current_msg_type.append(msg.type)

        elif msg.type == 'control_change':
            if msg.control == 64:  # 페달 정보 업데이트
                current_pedal = msg.value

        elif msg.type == 'set_tempo':
            current_tempo = mido.tempo2bpm(msg.tempo)  # 템포 정보 업데이트

        current_ticks = msg.time

        current_time += msg.time

    # 마지막 노트들을 처리
    if current_notes:
        midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, '', current_notes, current_velocities, current_pedal, current_tempo]], columns=columns)], ignore_index=True)

    return midi_df

In [6]:
result_df = midi_to_dataframe('midi_data/Fugue1.mid')
result_df.to_csv('hello1.csv', index=False)

print(result_df)

            Sec     Ticks                      msg_type Channel         Notes  \
0             0         0                            []       1            []   
1       0.46875   0.46875                     [note_on]       1          [60]   
2        0.9375         0           [note_off, note_on]       1      [60, 62]   
3       1.40625         0           [note_off, note_on]       1      [62, 64]   
4         1.875         0           [note_off, note_on]       1      [64, 65]   
..          ...       ...                           ...     ...           ...   
766   99.981427         0  [note_off, note_on, note_on]       3  [74, 79, 76]   
767   102.96169  2.980262                    [note_off]       1          [48]   
768  103.001164  0.039474                    [note_off]       0          [60]   
769  103.119585  0.118421                    [note_off]       1          [84]   
770  103.139321         0          [note_off, note_off]              [76, 79]   

       Velocities Pedal    

In [7]:
result_df = midi_to_dataframe('midi_data/Fugue3.mid')
result_df.to_csv('hello2.csv', index=False)

print(result_df)

            Sec     Ticks                                           msg_type  \
0             0         0                                                 []   
1       1.09756   1.09756                                          [note_on]   
2      1.463414         0                                [note_off, note_on]   
3      1.646341         0                                [note_off, note_on]   
4      1.829268         0                                [note_off, note_on]   
..          ...       ...                                                ...   
893  159.584519         0                     [note_off, note_off, note_off]   
894  159.723864  0.098361               [note_on, note_on, note_on, note_on]   
895  160.094554         0  [note_off, note_off, note_off, note_off, note_...   
896  160.137657         0      [note_on, note_on, note_on, note_on, note_on]   
897  162.206623         0  [note_off, note_off, note_off, note_off, note_...   

    Channel                 Notes      

In [8]:
result_df = midi_to_dataframe('midi_data/sample_midi.mid')
result_df.to_csv('hello3.csv', index=False)

print(result_df)

          Sec      Ticks                                       msg_type  \
0    2.748438   0.748438  [note_on, note_on, note_on, note_on, note_on]   
1    3.131250   0.382812                                     [note_off]   
2    3.139063   0.007812                                     [note_off]   
3    3.146875   0.007812                                     [note_off]   
4    3.150000   0.003125                                     [note_off]   
..        ...        ...                                            ...   
84  23.029688   0.029688  [note_on, note_on, note_on, note_on, note_on]   
85  25.315625   2.285938                                     [note_off]   
86  25.321875   0.000000                           [note_off, note_off]   
87  25.329688   0.007812                                     [note_off]   
88  59.000000  33.643750                                     [note_off]   

   Channel                 Notes            Velocities Pedal  Tempo  
0        0  [41, 53, 57, 60, 

In [10]:
result_df = midi_to_dataframe('midi_data/sample_midi.mid')
result_df.to_csv('hello4.csv', index=False)

print(result_df)

          Sec      Ticks                                       msg_type  \
0    2.748438   0.748438  [note_on, note_on, note_on, note_on, note_on]   
1    3.131250   0.382812                                     [note_off]   
2    3.139063   0.007812                                     [note_off]   
3    3.146875   0.007812                                     [note_off]   
4    3.150000   0.003125                                     [note_off]   
..        ...        ...                                            ...   
84  23.029688   0.029688  [note_on, note_on, note_on, note_on, note_on]   
85  25.315625   2.285938                                     [note_off]   
86  25.321875   0.000000                           [note_off, note_off]   
87  25.329688   0.007812                                     [note_off]   
88  59.000000  33.643750                                     [note_off]   

   Channel                 Notes            Velocities Pedal  Tempo  
0        0  [41, 53, 57, 60, 